In [ ]:
import kfp

# the namespace in which you deployed Kubeflow Pipelines
namespace = "kubeflow" 

client = kfp.Client(host=f"http://ml-pipeline-ui.{namespace}")

print(client.list_experiments())

In [14]:
# import all the deps
from typing import NamedTuple
from kfp import dsl

In [24]:
from typing import NamedTuple
from collections import namedtuple

@dsl.component(
    base_image="python:3.9",
    packages_to_install=["tensorflow", "minio", "numpy"]
)
def get_data_batch() -> NamedTuple(
    'Outputs',
    [
        ('datapoints_training', float),
        ('datapoints_test', float),
        ('dataset_version', str)
    ]
):
    import numpy as np
    from minio import Minio

    minio_client = Minio(
        "minio-service.kubeflow.svc.cluster.local:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )

    minio_bucket = "mlpipeline"

    print("[DEBUG] Attempting to download mnist.npz from Minio...")
    try:
        minio_client.fget_object(
            minio_bucket,
            "mnist.npz",
            "/tmp/mnist.npz"
        )
        print("[DEBUG] Downloaded mnist.npz successfully!")
    except Exception as e:
        print(f"[ERROR] Failed to download mnist.npz: {e}")
        raise

    def load_data():
        print("[DEBUG] Attempting to load /tmp/mnist.npz...")
        try:
            with np.load("/tmp/mnist.npz", allow_pickle=True) as f:
                x_train, y_train = f["x_train"], f["y_train"]
                x_test, y_test = f["x_test"], f["y_test"]
            print("[DEBUG] Loaded mnist.npz into memory successfully.")
        except Exception as e:
            print(f"[ERROR] Could not load /tmp/mnist.npz: {e}")
            raise
        return (x_train, y_train), (x_test, y_test)

    print("[DEBUG] Loading data from file...")
    try:
        (x_train, y_train), (x_test, y_test) = load_data()
        print("[DEBUG] Data loaded successfully!")
    except Exception as e:
        print(f"[ERROR] Data loading failed: {e}")
        raise

    # Save to numpy file, store in Minio
    try:
        print("[DEBUG] Saving x_train...")
        np.save("/tmp/x_train.npy", x_train)
        print("[DEBUG] Uploading x_train to Minio...")
        minio_client.fput_object(minio_bucket, "x_train", "/tmp/x_train.npy")
        print("[DEBUG] x_train upload complete.")

        print("[DEBUG] Saving y_train...")
        np.save("/tmp/y_train.npy", y_train)
        print("[DEBUG] Uploading y_train to Minio...")
        minio_client.fput_object(minio_bucket, "y_train", "/tmp/y_train.npy")
        print("[DEBUG] y_train upload complete.")

        print("[DEBUG] Saving x_test...")
        np.save("/tmp/x_test.npy", x_test)
        print("[DEBUG] Uploading x_test to Minio...")
        minio_client.fput_object(minio_bucket, "x_test", "/tmp/x_test.npy")
        print("[DEBUG] x_test upload complete.")

        print("[DEBUG] Saving y_test...")
        np.save("/tmp/y_test.npy", y_test)
        print("[DEBUG] Uploading y_test to Minio...")
        minio_client.fput_object(minio_bucket, "y_test", "/tmp/y_test.npy")
        print("[DEBUG] y_test upload complete.")
    except Exception as e:
        print(f"[ERROR] Saving or uploading npy files failed: {e}")
        raise

    dataset_version = "1.0"

    print(f"[INFO] x_train shape: {x_train.shape}")
    print(f"[INFO] y_train shape: {y_train.shape}")
    print(f"[INFO] x_test shape: {x_test.shape}")
    print(f"[INFO] y_test shape: {y_test.shape}")

    Outputs = namedtuple('Outputs', ['datapoints_training', 'datapoints_test', 'dataset_version'])
    result = Outputs(float(x_train.shape[0]), float(x_test.shape[0]), dataset_version)
    print(f"[INFO] Returning: {result}")
    return result

In [25]:
@dsl.component
def train_model():
    print("Train model")

/opt/conda/lib/python3.11/site-packages/kfp/dsl/component_decorator.py:126: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.9' to 'python:3.10' on Oct 1, 2025. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.10.
  return component_factory.create_component_from_func(


In [31]:
# pipeline 

@dsl.pipeline(
    name='digits-recognizer-pipeline',
    description='detect digits'
)
def output_test():
    step_1 = get_data_batch()
    step_2 = train_model().after(step_1)


In [33]:
# compile kfp 
if __name__ == "__main__":
    client = kfp.Client()

    arguments = {
        "no_epochs" : 1,
        "optimizer": "adam"
    }

    run_directly = 1
    
    if (run_directly == 1):
        client.create_run_from_pipeline_func(output_test,experiment_name="test")
    else: 
        print("else")
